In [ ]:
# This is necessary to recognize the modules
import os
import sys
import warnings

import pandas as pd
from web3.pm import BATCH_SIZE

from core.data_sources import CLOBDataSource

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_path)

In [ ]:
CONNECTOR_NAME = "binance_perpetual"
INTERVAL = "15m"
DAYS = 30
VOLATILITY_WINDOW = 50
VOLUME_WINDOW = 50
DEPTH_LEVELS = 10

In [ ]:
from core.data_sources.clob import CLOBDataSource

clob = CLOBDataSource()

In [ ]:
trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_rules

In [ ]:
trading_pairs = trading_rules.filter_by_quote_asset("USDT").get_all_trading_pairs()
trading_pairs

In [ ]:
candles = await clob.get_candles_last_days(CONNECTOR_NAME, trading_pairs[0], INTERVAL, DAYS)

In [ ]:
candles.plot(type="returns")

In [ ]:
BATCH_SIZE = 40
SLEEP_INTERVAL = 2

candles = await clob.get_candles_batch_last_days(CONNECTOR_NAME, trading_pairs, INTERVAL, DAYS, BATCH_SIZE, SLEEP_INTERVAL)

In [ ]:
candles[0].data

In [ ]:
import asyncio


async def get_imbalance(trading_pair):
    connector = clob.get_connector(CONNECTOR_NAME)
    ob = await connector._orderbook_ds._order_book_snapshot(trading_pair)
    cum_asks_volume = sum([row.amount for row in ob.asks[:DEPTH_LEVELS]])
    cum_bids_volume = sum([row.amount for row in ob.bids[:DEPTH_LEVELS]])
    # Compute bid ask ratio imbalance
    return (cum_bids_volume - cum_asks_volume) / (cum_bids_volume + cum_asks_volume)


n_trading_pairs = len(trading_pairs)
batches = 2
all_imbalances = []
for i in range(batches):
    start = i * BATCH_SIZE
    end = (i + 1) * BATCH_SIZE
    tasks = [get_imbalance(trading_pair) for trading_pair in trading_pairs[start:end]]
    imbalances = await asyncio.gather(*tasks)
    all_imbalances.extend(imbalances)
    await asyncio.sleep(10.0)

In [ ]:
len(trading_pairs)

In [ ]:
import numpy as np


def get_volatility(df, window):
    df["log_return"] = np.log(df["close"] / df["close"].shift(1))
    df["volatility"] = df["log_return"].rolling(window=window).std() * np.sqrt(window)
    return df["volatility"].iloc[-1]


def get_volume_imbalance(df, window):
    # Calculate volume metrics
    df["volume_usd"] = df["volume"] * df["close"]
    df["buy_taker_volume_usd"] = df["taker_buy_base_volume"] * df["close"]
    df["sell_taker_volume_usd"] = df["volume_usd"] - df["buy_taker_volume_usd"]
    # Calculate buy/sell imbalance
    df["buy_sell_imbalance"] = df["buy_taker_volume_usd"] - df["sell_taker_volume_usd"]
    # Calculate rolling total volume
    rolling_total_volume_usd = df["volume_usd"].rolling(window=window, min_periods=1).sum()
    return rolling_total_volume_usd.iloc[-1]


async def get_imbalance(trading_pair):
    connector = clob.get_connector(CONNECTOR_NAME)
    ob = await connector._orderbook_ds._order_book_snapshot(trading_pair)
    cum_asks_volume = sum([row.amount for row in ob.asks[:DEPTH_LEVELS]])
    cum_bids_volume = sum([row.amount for row in ob.bids[:DEPTH_LEVELS]])
    # Compute bid ask ratio imbalance
    return (cum_bids_volume - cum_asks_volume) / (cum_bids_volume + cum_asks_volume)


report = []
for candle in candles:
    trading_pair = candle.trading_pair
    df = candle.data
    # Calculate logarithmic returns
    volatility = get_volatility(df, VOLATILITY_WINDOW)
    volume_imbalance = get_volume_imbalance(df, VOLUME_WINDOW)
    # ADD METHOD FOR ORDER BOOK IMBALANCE
    report.append({"trading_pair": trading_pair, "volatility": volatility, "volume_imbalance": volume_imbalance})

In [ ]:
pd.DataFrame(report)